# Explore the data 

## Import train_prompts data set

In [ ]:
import pandas as pd
train_pro=pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv')
train_pro

In [ ]:
train_pro['instructions'][0]

In [ ]:
train_pro['instructions'][1]

In [ ]:
# import pandas as pd

# # Assuming your DataFrame is named 't'
# t = pd.read_csv('/kaggle/input/llm-generated-text-dataset/train_drcat_04.csv')

# # Select 15,000 rows for label=0
# generated_0_sample = t[t['label'] == 0].sample(n=15000, random_state=42)

# # Select all rows for label=1
# generated_1_sample = t[t['label'] == 1]

# # Concatenate the two samples into a new DataFrame
# train2 = pd.concat([generated_0_sample, generated_1_sample])

# # Shuffle the new DataFrame
# train2 = train2.sample(frac=1, random_state=42).reset_index(drop=True)

# print(train2['label'].value_counts())
# print(train2.info())
# print(train2.shape)
# print(train2.head())
# print(train2.columns)


In [ ]:
# # Assuming your DataFrame is named 'train2'
# train2.drop(columns=['essay_id','prompt' ,'fold'],inplace=True)
# train2 = train2[train2['source'] != 'train_essays']



In [ ]:
# train2.rename(columns={'label': 'generated'}, inplace=True)
# train2.drop('source',axis=1,inplace=True)
# train2.head()

# Import data set hand written eassys from students only 3 genereted from LLM

In [ ]:
train=pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')

### Explore and clean the train dataset

In [ ]:
train['id'] = pd.factorize(train['id'])[0] + 1
print(train.info())
train.head()

In [ ]:
train['text'][0]

In [ ]:
print(train.prompt_id.value_counts())
print(train.generated.value_counts())

In [ ]:
train[train['generated']==1]

## Importing dataset generated from LLM

In [ ]:
train1=pd.read_csv('/kaggle/input/llm-generated-essay-using-palm-from-google-gen-ai/LLM_generated_essay_PaLM.csv')

### Explore and clean the data

In [ ]:
train1['prompt_id']=train1['prompt_id'].astype('int64')
train1['generated']=train1['generated'].astype('int64')
print(train1.info())
train1.head()

In [ ]:
print(train1.prompt_id.value_counts())
print(train1.generated.value_counts())

In [ ]:
train1['text'][5]

## Concatination
### hand written and LLM generated datasets

In [ ]:
df=pd.concat([train,train1],ignore_index=True)
df = df.sample(frac=1,random_state=42).reset_index(drop=True).drop(columns=['id','prompt_id']) 
print(df.generated.value_counts())
print(df.info())
df.head()

In [ ]:
df['text']=df['text'].str.lower()
df.head()

In [ ]:
df['text'][0]

# Preparing the data

In [ ]:
# !pip install bert-for-tf2
# # !pip install ipywidgets

## Removing Special Charectors

In [ ]:
import re

df['text'] = [re.sub("[.,|:='~^0-9\\\%;&')`@(-_]", "", item) for item in df['text']]
df.head()

## Seperate tokens to text column

In [ ]:
# !pip install nltk

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
df['text']= [word_tokenize(item) for item in df['text']]
df.head()

## Removing STOP WORDS 

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

#Remove them from our dataframe
df['text'] = df['text'].apply(lambda x: [i for i in x if i not in stop])

df.head()

In [ ]:
from nltk.corpus import stopwords
stopwords.words('english')

## Stemming

In [ ]:
import nltk
from nltk.stem import PorterStemmer
#nltk.download("punkt")

# Initialize Python porter stemmer
ps = PorterStemmer()

# Perform stemming and update the 'text' column
df['text'] = df['text'].apply(lambda text_list: ' '.join([ps.stem(word) for word in ' '.join(text_list).split(' ')]))



## Converting tokens to string form

In [ ]:
df['text'] = df['text'].apply(lambda x: ''.join(' '.join(word) for word in x))
df.head()


In [ ]:
df['text'][0]

In [ ]:
max_len=min(df['text'].apply(lambda x : len(x)))
max_len

In [ ]:
# from transformers import BertTokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
from transformers import BertTokenizer

# Save the tokenizer to a directory

#tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
saved_directory = '/kaggle/input/bert/keras/bert-based-cased-nlp/1'
#tokenizer.save_pretrained(saved_directory)

# Load the tokenizer from the saved directory
tokenizer = BertTokenizer.from_pretrained(saved_directory)


In [ ]:
# import pandas as pd

# def get_dataset_partitions_tf(df, train_split=0.92, val_split=0.08, shuffle=True, shuffle_size=10000):
#     assert (round(train_split + val_split) == 1)
    
#     df_size = len(df)
    
#     if shuffle:
#         df = df.sample(frac=1, random_state=12)  # Shuffle the DataFrame
    
#     train_size = int(train_split * df_size)
#     val_size = int(val_split * df_size)
    
#     train_df = df.iloc[:train_size]
#     val_df = df.iloc[train_size:train_size + val_size]
    
#     return train_df, val_df



In [ ]:
# df, test_ds = get_dataset_partitions_tf(df)
# print(len(df))
# print(len(test_ds))

In [ ]:
# text=test_ds['text']
# labels=test_ds['generated']

## Convert data into numerical form

In [ ]:
seq_length=635
tokens=tokenizer(df['text'].tolist(),
                max_length=seq_length,
                truncation=True,
                padding='max_length',
                add_special_tokens=True,
                return_tensors='np')

## Labels for bert model

In [ ]:
num_samples=len(df)
import numpy as np
arr=df['generated'].values
labels = np.zeros((num_samples, arr.max()+1))
labels[np.arange(num_samples), arr] = 1

In [ ]:
labels

In [ ]:
df_ids = tokens['input_ids']
df_mask = tokens['attention_mask']

In [ ]:
len(df_ids[0])

In [ ]:
len(df_mask)

## Final dataset for Bert model

In [ ]:
import tensorflow as tf
dataset=tf.data.Dataset.from_tensor_slices((df_ids,df_mask,labels))
dataset.take(1)

In [ ]:
def map_func(input_ids, masks, labels):
    return {'input_ids': input_ids,
           'attention_mask': masks}, labels

In [ ]:
dataset = dataset.map(map_func)
dataset.take(1)

In [ ]:
batch_size=10
dataset=dataset.shuffle(10000, seed=42).batch(batch_size,drop_remainder=True)

In [ ]:
dataset.take(1)

## Train test split

In [ ]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.2, shuffle=True, shuffle_size=10000):
    assert (train_split + val_split) == 1
    
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    
    return train_ds, val_ds,

In [ ]:
train_ds, val_ds = get_dataset_partitions_tf(dataset)

In [ ]:
print(len(train_ds))
print(len(val_ds))

In [ ]:
train_ds

## Model

In [ ]:
from transformers import TFAutoModel
from transformers import TFBertModel


In [ ]:
# !wget https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-tf_model.h5

## Fine tune the TFAutoModel

In [ ]:
bert_local_path='/kaggle/input/bert/keras/bert-based-cased-nlp/1'


In [ ]:
input_ids = tf.keras.layers.Input(shape=(635,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(635,), name='attention_mask', dtype='int32')

bert = TFAutoModel.from_pretrained(bert_local_path)
# transformer
embeddings = bert.bert(input_ids, attention_mask=mask)[1]

# classifier head
x = tf.keras.layers.Dense(1024, activation='relu')(embeddings)
y = tf.keras.layers.Dense(2, activation='softmax', name='outputs')(x)


model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss=tf.keras.losses.BinaryCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=2
)


## Saving the model

In [ ]:
model.save("ai_generate.tf")

## Loading model

In [ ]:
loaded_model = tf.keras.models.load_model("/kaggle/working/ai_generate.tf")


## Import test data

In [ ]:
test=pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
test.head()

In [ ]:
test.info()

## Prepare test dataset

#### Function for preprocess test data

In [ ]:
def preparation(data):
    # string to lower case
    data = data.lower()
    # removing special characters
    data = re.sub("[.,|:='~^0-9\\\%;&')`@(-_]", " ", data)
    # tokenizing
    data = word_tokenize(data)
    # removing stopwords
    data = [word for word in data if word not in stop]
    # converting tokens to string
    data = ''.join(data)
    return data

# Apply the preparation function to the 'text' column in both training and test data
test['text'] = test['text'].apply(preparation)


#### Function for Tokenizing test data

In [ ]:
def prep_data(text):
    tokens=tokenizer(text,
                max_length=seq_length,
                truncation=True,
                padding='max_length',
                add_special_tokens=True,
                return_tensors='np')
    
    df_ids = tokens['input_ids']
    df_mask = tokens['attention_mask']
    
    # tokenizer returns int32 tensors, we need to return float64, so we use tf.cast
    return df_ids, df_mask


#### Apply preprocess test data function to test data and make a new list pro

In [ ]:
pro=[]
for t in test['text']:
    probs=prep_data(t)
    pro.append(probs)

#### Apply tokenizer function to preproessed test data(pro) and make a new list submission

In [ ]:
submission=[]
for m in pro:
    pre=model.predict(m)
    print(pre)
    submission.append(m)


#### Extract binary predictions from model

In [ ]:
submiss = []
for m in pro:
    pre = loaded_model.predict(m)
    print(pre)
    sub=np.argmax(pre)
    submiss.append(sub)
    

# submission_values = [pred[0] for pred in submiss]
# print(submission_values)
submiss

#### Making the predictions into desired submission format

In [ ]:
# new_list = []

# for pred in submission_values:
#     if pred[0] > 0.5:
#         new_value = np.round(pred[0] / 2, 1)
#     else:
#         new_value = round(pred[1],1)
#     new_list.append(new_value)

# print(new_list)


## Making submission dataframe

In [ ]:
id=test['id']
id

In [ ]:
submission=pd.DataFrame({'id':id})
submission

In [ ]:
submission['generated']=submiss
submission

## submission.csv file

In [ ]:
submission.to_csv('submission.csv',index=False)